In [1]:
import sys
import importlib
sys.path.append(r'../../pykneer')
sys.path.append(r'../pykneer')
import pykneer_io as io
importlib.reload(io)
import preprocessing_for_nb as prep
importlib.reload(prep)
import segmentation_sa_for_nb as segm
importlib.reload(segm)

import SimpleITK as sitk
import matplotlib.pyplot as plt

from preparing_images import prepare_image_and_list

In [2]:
# process list
# first in TP as reference
caselist = [
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9939718','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9939718','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9941231','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9941231','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9942724','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9942724','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9941446','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9941446','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9404954','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9404954','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9942030','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9942030','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
]

prepare_image_and_list(caselist)

ahaknee0tp29  exist
dcm path for TPid 0 Z:/OAI Baseline/ge0/9189303/10397606
Loading QVJ Y:/0tp29\cascade/0/9189303/10397606L\E10397606L_L.QVJ
Adding QVS E10397606LS101_L.QVS
ahaknee12tp29  exist
dcm path for TPid 1 Z:/OAI 12 Months/2.C.2/9189303/20060522/11151205
Loading QVJ Y:/12tp29\cascade/12/9189303/20060522L\E20060522L_L.QVJ
Adding QVS E20060522LS101_L.QVS
ahaknee18tp29  exist
cannot find dcm path for TPid 2
ahaknee24tp29  exist
cannot find dcm path for TPid 3
ahaknee30tp29  exist
cannot find dcm path for TPid 4
ahaknee36tp29  exist
cannot find dcm path for TPid 5
ahaknee48tp29  exist
cannot find dcm path for TPid 6
ahaknee72tp29  exist
cannot find dcm path for TPid 8
ahaknee96tp29  exist
cannot find dcm path for TPid 10
ahaknee0tp29  exist
dcm path for TPid 0 Z:/OAI Baseline/ge0/9189303/10397613
Loading QVJ Y:/0tp29\cascade/0/9189303/10397613R\E10397613R_R.QVJ
Adding QVS E10397613RS101_R.QVS
ahaknee12tp29  exist
dcm path for TPid 1 Z:/OAI 12 Months/2.C.2/9189303/20060522/1115121

Adding QVS E20080724LS101_L.QVS
ahaknee72tp29  exist
cannot find dcm path for TPid 8
ahaknee96tp29  exist
dcm path for TPid 10 Z:/OAI 96 Months/10.C.1/9942724/20130508/13383008
Loading QVJ Y:/96tp29\cascade/96/9942724/20130508L\E20130508L_L.QVJ
Adding QVS E20130508LS101_L.QVS
ahaknee0tp29  exist
dcm path for TPid 0 Z:/OAI Baseline/gc0/9942724/10119313
Loading QVJ Y:/0tp29\cascade/0/9942724/10119313R\E10119313R_R.QVJ
Adding QVS E10119313RS101_R.QVS
ahaknee12tp29  exist
dcm path for TPid 1 Z:/OAI 12 Months/1.C.2/9942724/20051011/10198011
Loading QVJ Y:/12tp29\cascade/12/9942724/20051011R\E20051011R_R.QVJ
Adding QVS E20051011RS101_R.QVS
ahaknee18tp29  exist
cannot find dcm path for TPid 2
ahaknee24tp29  exist
dcm path for TPid 3 Z:/OAI 24 Months/3.C.2/9942724/20061024/10371314
Loading QVJ Y:/24tp29\cascade/24/9942724/20061024R\E20061024R_R.QVJ
Adding QVS E20061024RS101_R.QVS
ahaknee30tp29  exist
cannot find dcm path for TPid 4
ahaknee36tp29  exist
dcm path for TPid 5 Z:/OAI 36 Months/5.C.

dcm path for TPid 1 Z:/OAI 12 Months/1.C.2/9942030/20060627/11208212
Loading QVJ Y:/12tp29\cascade/12/9942030/20060627R\E20060627R_R.QVJ
Adding QVS E20060627RS101_R.QVS
ahaknee18tp29  exist
cannot find dcm path for TPid 2
ahaknee24tp29  exist
dcm path for TPid 3 Z:/OAI 24 Months/3.C.2/9942030/20070820/11940413
Loading QVJ Y:/24tp29\cascade/24/9942030/20070820R\E20070820R_R.QVJ
Adding QVS E20070820RS101_R.QVS
ahaknee30tp29  exist
cannot find dcm path for TPid 4
ahaknee36tp29  exist
dcm path for TPid 5 Z:/OAI 36 Months/5.C.1/9942030/20080819/12486512
Loading QVJ Y:/36tp29\cascade/36/9942030/20080819R\E20080819R_R.QVJ
Adding QVS E20080819RS101_R.QVS
ahaknee48tp29  exist
dcm path for TPid 6 Z:/OAI 48 Months/6.C.1/9942030/20090818/13195913
Loading QVJ Y:/48tp29\cascade/48/9942030/20090818R\E20090818R_R.QVJ
Adding QVS E20090818RS101_R.QVS
ahaknee72tp29  exist
cannot find dcm path for TPid 8
ahaknee96tp29  exist
cannot find dcm path for TPid 10


In [3]:
def preprocess():
    input_file_name           = ".\image_list_preprocessing.txt"
    n_of_cores                = 1 # change the number of cores according to your computer
    intensity_standardization = 0

    # Image information
    image_data = io.load_image_data_preprocessing(input_file_name)

    ref = sitk.ReadImage('../../reference/newsubject/reference.mha')

    # Spatial standardization
    # Step 0: Read the dicom images
    prep.read_dicom_stack(image_data, n_of_cores)
    # Save image header
    prep.print_dicom_header(image_data, n_of_cores)    # save to *.txt

    # Step 1: Change orientation to RAI
    #prep.axial_to_sagittal(image_data, n_of_cores)
    prep.orientation_to_rai(image_data, n_of_cores)

    # Step 2: Change laterality if knee is right
    prep.flip_rl(image_data, n_of_cores)
    # Step 3: Set image origin to (0,0,0)
    prep.origin_to_zero(image_data, n_of_cores)    # Images are saved as *_orig.mha and they are anonymized

    # 4. Correct magnetic field inhomogeneities
    # Magnetic fields inhomogeneities create grey shades on images. This correction removes these shades. 
    # This is the longest step of the processing. It can take up to 15-20 min on a standard PC or laptop
    if intensity_standardization == 1:
        prep.field_correction(image_data, n_of_cores)
        prep.rescale_to_range(image_data, n_of_cores)  # 5. Rescale intensities to [0 100]
        prep.edge_preserving_smoothing(image_data, n_of_cores)  # 6. Edge preserving smoothing

# preprocess()

In [3]:
def segmentation_sa():
    input_file_name = "./image_list_newsubject.txt"
    modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
    n_of_cores      = 1

    image_data = io.load_image_data_segmentation(modality, input_file_name)
    segm.prepare_reference(image_data)
    ### NOTE!!! reference_fv is dummy!!!

    # Segment Bone
#     # 1. Register image to reference
    segm.register_bone_to_reference(image_data, n_of_cores)
#     # 2. Invert transformations
    segm.invert_bone_transformations(image_data, n_of_cores)
#     # 3. Warp reference mask to moving image
    segm.warp_bone_mask(image_data, n_of_cores)
    
    segm.register_tibia_to_reference(image_data, n_of_cores)
    segm.invert_tibia_transformations(image_data, n_of_cores)
    segm.warp_tibia_mask(image_data, n_of_cores)

    # Segment cartilage
    segm.register_cartilage_to_reference(image_data, n_of_cores)
    segm.invert_cartilage_transformations(image_data, n_of_cores)
    segm.warp_cartilage_mask(image_data, n_of_cores)
    
# segmentation_sa()

In [13]:
sys.path.append("../../..")
from MyProcessDicom import *
input_file_name = "./image_list_newsubject.txt"
modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 1

image_data = io.load_image_data_segmentation(modality, input_file_name)

-> image information loaded


In [ ]:
image_data[0]

In [17]:
sys.path.append("../../..")
from MyProcessDicom import *
input_file_name = "./image_list_reference.txt"
modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 1

image_data = io.load_image_data_segmentation(modality, input_file_name)

-> image information loaded


In [5]:
def segmentation_sa_long():
    input_file_name = "./image_list_longitudinal.txt"
    modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
    n_of_cores      = 1

    image_data = io.load_image_data_segmentation(modality, input_file_name)
    segm.prepare_reference(image_data)

    segm.register_bone_to_reference(image_data, n_of_cores)
    segm.invert_bone_transformations(image_data, n_of_cores)
    segm.warp_tibia_mask(image_data, n_of_cores)
    segm.warp_bone_mask(image_data, n_of_cores)

    # segm.register_vessel_to_reference(image_data, n_of_cores)
    # segm.invert_vessel_transformations(image_data, n_of_cores)
    # segm.warp_vessel_mask(image_data, n_of_cores)
    
#     segm.register_cartilage_to_reference(image_data, n_of_cores)
#     segm.invert_cartilage_transformations(image_data, n_of_cores)
#     segm.warp_cartilage_mask(image_data, n_of_cores)
    
# segmentation_sa_long()

In [ ]:
# preprocess()
# segmentation_sa()
# segmentation_sa_long()

In [19]:
sys.path.append("../../..")
from MyProcessDicom import *
input_file_name = "./image_list_longitudinal.txt"
modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
n_of_cores      = 1

image_data = io.load_image_data_segmentation(modality, input_file_name)

-> image information loaded


In [ ]:
image_data[0]

In [6]:
import matplotlib.pyplot as plt
import os
%matplotlib inline
import numpy as np
from ipywidgets import interact, fixed
def show_sagittal_images(image, length, index, tp_str):
    
    image_npa = sitk.GetArrayViewFromImage(image)
    size = np.size(image_npa, 2)
    # Draw the fixed image in the first subplot.
    plt.subplot(length,1,index)
    plt.imshow(image_npa[:,:,size//2],cmap=plt.cm.Greys_r);
    plt.title(tp_str)
    plt.axis('off')

In [7]:
def show_slice_images(all_image_data):
    last_name = '0000000L'
    # for each image
    for i in range(0, len(all_image_data)):

        # get paths and file names of the current image
        image_data                    = all_image_data[i]
#         if image_data['moving_name'][:12] != last_name:
#             last_name = image_data['moving_name'][:8]
        print(image_data['moving_name'][:12])
            
        moving_file_name              = image_data['registered_sub_folder']    + image_data['fspline_name']
        reference_file_name              = image_data['reference_folder']    + image_data['reference_name']
    #     image_data = all_image_data[0]
    #     reference_file_name = image_data['registered_sub_folder']    + image_data['frigid_name']
    #         mask_file_name                = image_data["segmented_folder"] + image_data[anatomy + "mask"]

    #     test_name          = image_data["reference_folder"] + image_data["reference_name"]
        # read the images
        reference = sitk.ReadImage(reference_file_name)
        moving = sitk.ReadImage(moving_file_name)
    #         mask   = sitk.ReadImage(mask_file_name)
    #     test = sitk.ReadImage(test_name)

    #     reference = tmp_resample(reference, moving)
    #     print(moving.GetSize())

        # images from simpleitk to numpy
    #     reference_py = sitk.GetArrayFromImage(reference)
    #     moving_py = sitk.GetArrayFromImage(moving)
    #         mask_py   = sitk.GetArrayFromImage(mask)
        show_mixed_images(reference, moving)
    #     show_mixed_images(reference, reference)

In [8]:
def display_images_with_mask(image_z, fixed, moving):
    # img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z]
    fixed = sitk.GetArrayFromImage(fixed)
    moving = sitk.GetArrayFromImage(moving) # mask
    dst = fixed[image_z,:,:]*0.5+moving[image_z,:,:]*0.5*255
    plt.figure(figsize=(9,9))
    plt.imshow(dst, cmap=plt.cm.Greys_r)
    # plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

def show_masked_images(image, mask):
    interact(display_images_with_mask, image_z=(0,image.GetSize()[2] - 1), 
             fixed = fixed(image), moving=fixed(mask));

In [9]:
def display_images_with_mask_sagittal(image_z, fixed, moving):
    # img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z]
    fixed = sitk.GetArrayFromImage(fixed)
    moving = sitk.GetArrayFromImage(moving) # mask
    dst = fixed[:,:,image_z]*0.5+moving[:,:,image_z]*0.5*255
    plt.figure(figsize=(7,7))
    plt.imshow(dst, cmap=plt.cm.Greys_r)
    # plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

def show_masked_images_sagittal(image, mask):
    interact(display_images_with_mask_sagittal, image_z=(0,image.GetSize()[0] - 1), 
             fixed = fixed(image), moving=fixed(mask));
    
def display_images_with_all_masks(image_z, alpha, fixed, moving, tibia):
    # img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z]
    fixed = sitk.GetArrayFromImage(fixed)
    moving = sitk.GetArrayFromImage(moving) # mask
    tibia = sitk.GetArrayFromImage(tibia) # mask
    dst = fixed[:,:,image_z]*alpha+moving[:,:,image_z]*(1-alpha)*255+tibia[:,:,image_z]*(1-alpha)*255
    dst = np.flipud(dst)
    plt.figure(figsize=(7,7))
    plt.imshow(dst, cmap=plt.cm.Greys_r)
    # plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()

def show_all_masked_images(image, mask, tibia):
    interact(display_images_with_all_masks, image_z=(0,image.GetSize()[0] - 1), alpha=(0.0,1.0,0.05),
             fixed = fixed(image), moving=fixed(mask), tibia=fixed(tibia));

In [62]:
# mask_file_name = image_data[0]['reference_folder'] + '9941231L_TP0_prep_f.mha'
mask_file_name = image_data[0]['segmented_folder'] + '9404954L_TP3_prep_f.mha'
# mask_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\newsubject\\reference_f.mha"
tibia_file_name = image_data[0]['segmented_folder'] + '9404954L_TP3_prep_t_rigid.mha'
# tibia_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9404954L_TP3_prep_t.mha"
# mask_file_name = 'C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9941446L_TP0_prep_t.mha'
moving_file_name   = image_data[0]['moving_folder']    + "9404954L_TP3_prep.mha"
# moving_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\registered\\9404954L_TP1_prep\\t_rigid.mha"
mask = sitk.ReadImage(mask_file_name)
tibia = sitk.ReadImage(tibia_file_name)
moving = sitk.ReadImage(moving_file_name)
key = 1
if key == 1:
    mask = resample_bwimage(mask)
    tibia = resample_bwimage(tibia)
    moving = resample_bwimage(moving)
#     show_masked_images_sagittal(moving, mask)
    show_all_masked_images(moving, mask,tibia)
else:
#     show_all_masked_images(moving, mask,tibia)
    show_masked_images(moving, tibia)    

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [10]:
def show_sagital_slice_images(all_image_data, is_rigid=False):
    last_name = '0000000L'
    # for each image
    for i in range(0, len(all_image_data)):
#     for i in range(0, 4):

        # get paths and file names of the current image
        image_data                    = all_image_data[i]
#         if image_data['moving_name'][:8] != last_name:
#             last_name = image_data['moving_name'][:8]
        print(image_data['moving_name'][:12])       
   
        # mask_file_name = image_data[0]['reference_folder'] + '9941231L_TP0_prep_f.mha'
        if is_rigid:
            mask_file_name = image_data['segmented_folder'] + image_data['fmask'][:-5] + 'f_rigid.mha'
            tibia_file_name = image_data['segmented_folder'] + image_data['fmask'][:-5] + 't_rigid.mha'
        else:
            mask_file_name = image_data['segmented_folder'] + image_data['fmask']
            tibia_file_name = image_data['segmented_folder'] + image_data['fmask'][:-5] + 't.mha'
        # mask_file_name = 'C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9941446L_TP0_prep_t.mha'
        moving_file_name   = image_data['moving_folder']    + image_data['moving_name']
        mask = sitk.ReadImage(mask_file_name)
        tibia = sitk.ReadImage(tibia_file_name)
        moving = sitk.ReadImage(moving_file_name)
        key = 1
        if key == 1:
            mask = resample_bwimage(mask)
            tibia = resample_bwimage(tibia)
            moving = resample_bwimage(moving)
        #     show_masked_images_sagittal(moving, mask)
            show_all_masked_images(moving, mask,tibia)
        else:
            show_masked_images(moving, mask)    

In [11]:
def show_sagital_reference(all_image_data):
    last_name = '0000000L'
    # for each image
    for i in range(0, len(all_image_data)):
#     for i in range(0, 4):

        # get paths and file names of the current image
        image_data                    = all_image_data[i]
#         if image_data['moving_name'][:8] != last_name:
#             last_name = image_data['moving_name'][:8]
        print(image_data['moving_name'][:12])       
   
        # mask_file_name = image_data[0]['reference_folder'] + '9941231L_TP0_prep_f.mha'
        mask_file_name = image_data['segmented_folder'] + image_data['fmask']
        tibia_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\" + image_data['fmask'][:-5] + 't.mha'
        # mask_file_name = 'C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9941446L_TP0_prep_t.mha'
        moving_file_name   = image_data['moving_folder']    + image_data['moving_name']
        mask = sitk.ReadImage(mask_file_name)
        tibia = sitk.ReadImage(tibia_file_name)
        moving = sitk.ReadImage(moving_file_name)
        key = 1
        if key == 1:
            mask = resample_bwimage(mask)
            tibia = resample_bwimage(tibia)
            moving = resample_bwimage(moving)
        #     show_masked_images_sagittal(moving, mask)
            show_all_masked_images(moving, mask,tibia)
        else:
            show_masked_images(moving, mask)    

In [18]:
show_sagital_reference(image_data)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [ ]:
# 9939718L_TP10, 9939718R_TP10, 9941231L_TP10, 9942724L_TP10, 9942724R_TP10,
# 9941446R_TP1, 9404954L_TP3, 9404954L_TP10

In [14]:
show_sagital_slice_images(image_data)

9189303L_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9939718L_TP0


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030L_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP0


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [16]:
# rigid_only
show_sagital_slice_images(image_data, is_rigid=True)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [ ]:
# 9939718L_TP10, 9941231L_TP10, 9942723L_TP10, 9942724R_TP10

In [20]:
show_sagital_slice_images(image_data)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [ ]:
# 9939718L_TP10

In [14]:
def get_max_coordinate(image):
    array = sitk.GetArrayFromImage(image)
    return np.max(np.nonzero(np.argmax(np.argmax(array, axis = 1), axis=1)))

def computeQualityMeasures(lP,lT):
    quality=dict()
#     labelPred=sitk.GetImageFromArray(lP, isVector=False)
#     labelTrue=sitk.GetImageFromArray(lT, isVector=False)
    labelPred = lP
    labelTrue = lT
    hausdorffcomputer=sitk.HausdorffDistanceImageFilter()
    hausdorffcomputer.Execute(labelTrue,labelPred)
    quality["avgHausdorff"]=hausdorffcomputer.GetAverageHausdorffDistance()
    quality["Hausdorff"]=hausdorffcomputer.GetHausdorffDistance()
 
    dicecomputer=sitk.LabelOverlapMeasuresImageFilter()
    dicecomputer.Execute(labelTrue,labelPred)
    quality["dice"]=dicecomputer.GetDiceCoefficient()
    
    quality["max_y_coordinate"]= get_max_coordinate(lP) - get_max_coordinate(lT)
 
    return quality

In [10]:
def compute_quality_metrics(all_image_data):
    last_name = '0000000L'
    # for each image
    for i in range(0, len(all_image_data)):
        image_data                    = all_image_data[i]
        print(image_data['moving_name'][:12])       
   
        reference_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\" + image_data['fmask'][:-5] + 't.mha'
        tibia_file_name = image_data['segmented_folder'] + image_data['fmask'][:-5] + 't_rigid.mha'
#         moving_file_name   = image_data['moving_folder']    + image_data['moving_name']
        ref = sitk.ReadImage(reference_file_name)
        tibia = sitk.ReadImage(tibia_file_name)
        ref = resample_bwimage(ref)
        tibia = resample_bwimage(tibia)
        quality = computeQualityMeasures(tibia,ref)
        print(quality)

In [15]:
compute_quality_metrics(image_data)

9941231L_TP1
{'avgHausdorff': 0.01485142618078511, 'Hausdorff': 2.9393621166479305, 'dice': 0.9733380493897071, 'max_y_coordinate': 0}
9941231L_TP3
{'avgHausdorff': 0.10891810536165775, 'Hausdorff': 4.809180066985015, 'dice': 0.9283905304922564, 'max_y_coordinate': -4}
9941231L_TP5
{'avgHausdorff': 0.8685235299566747, 'Hausdorff': 14.642444883043167, 'dice': 0.8257057521934366, 'max_y_coordinate': 0}
9941231L_TP6
{'avgHausdorff': 0.02280669503591616, 'Hausdorff': 4.043421071116263, 'dice': 0.9672709118725008, 'max_y_coordinate': 0}
9941231L_TP8
{'avgHausdorff': 0.1747890754034586, 'Hausdorff': 7.190700741431328, 'dice': 0.9016089849625193, 'max_y_coordinate': 0}
9941231L_TP1
{'avgHausdorff': 2.6053782848898805, 'Hausdorff': 25.468673459336664, 'dice': 0.5657377435315153, 'max_y_coordinate': 9}
9941231R_TP1
{'avgHausdorff': 0.07938921044232242, 'Hausdorff': 5.358253913358716, 'dice': 0.9297051786728612, 'max_y_coordinate': 0}
9941231R_TP3
{'avgHausdorff': 0.15155002188949465, 'Hausdorff

In [5]:
compute_quality_metrics(image_data)

9941231L_TP1
{'avgHausdorff': 0.016422185901815316, 'Hausdorff': 2.9618862936773946, 'dice': 0.9721427094716323}
9941231L_TP3
{'avgHausdorff': 0.11025311468186857, 'Hausdorff': 4.753580740991995, 'dice': 0.9269705666421945}
9941231L_TP5
{'avgHausdorff': 1.0987485698965545, 'Hausdorff': 17.503781327591614, 'dice': 0.8050918786050647}
9941231L_TP6
{'avgHausdorff': 0.03622442195657248, 'Hausdorff': 3.7358640511058487, 'dice': 0.952930229832898}
9941231L_TP8
{'avgHausdorff': 0.2306335566377987, 'Hausdorff': 8.727175815141857, 'dice': 0.8930787903228502}
9941231L_TP1
{'avgHausdorff': 2.4066532532259313, 'Hausdorff': 21.426813073170866, 'dice': 0.49642741667821416}
9941231R_TP1
{'avgHausdorff': 0.15526014375628483, 'Hausdorff': 6.450131711620856, 'dice': 0.9057088867280553}
9941231R_TP3
{'avgHausdorff': 0.20353252079709494, 'Hausdorff': 7.162919437014073, 'dice': 0.8970969770033942}
9941231R_TP5
{'avgHausdorff': 0.8741121656922207, 'Hausdorff': 16.07889566369858, 'dice': 0.8187640150215986}


In [50]:
def tmp_resample(fixed_image, reference):
    isoresample = sitk.ResampleImageFilter()
    isoresample.SetInterpolator(sitk.sitkBSpline)
    isoresample.SetOutputDirection(fixed_image.GetDirection())
    isoresample.SetOutputOrigin(fixed_image.GetOrigin())
    orig_spacing = fixed_image.GetSpacing()
    new_spacing = (orig_spacing[0],orig_spacing[0],orig_spacing[0])
    isoresample.SetOutputSpacing(new_spacing)
    orig_size = np.array(fixed_image.GetSize(), dtype=np.int)    
    new_size = orig_size.copy()
    new_size[2] = int(orig_size[2]*(orig_spacing[2]/orig_spacing[0])+0.5)
    new_size = [int(s) for s in new_size]
    print(orig_size, new_size)
    isoresample.SetSize(new_size)
    return isoresample.Execute(fixed_image)

In [29]:
def show_registered_images(find_str): 
    path = 'C:\\Zhixuan\\OAI-registration\\pyKNEEr-yg\\registered\\'
    file_path_list = []
    reference = sitk.ReadImage(path+find_str+"_TP0_prep\\f_rigid.mha")
    for file in os.listdir(path):
        if find_str in file:
            file_path_list.append(file)
    i = 1
    # Create a figure with two subplots and the specified size.
    plt.subplots(len(file_path_list), 1,figsize=(25,20))
    for tmp_path in file_path_list:
        result_file_name              = path + tmp_path + '\\f_rigid.mha'
#         mask_file_name                = image_data["segmented_folder"] + image_data[anatomy + "mask"]

        # read the images
        result = sitk.ReadImage(result_file_name)
#         mask   = sitk.ReadImage(mask_file_name)

        # images from simpleitk to numpy
    #     result_py = sitk.GetArrayFromImage(result)
    #         mask_py   = sitk.GetArrayFromImage(mask)
        pos1 = tmp_path.find('TP', 0)
        pos2 = tmp_path.find('_', pos1)
        tp = tmp_path[pos1:pos2]
        if tp != 'TP0':
            result = resample_image(result)
        
            
        show_sagittal_images(result, len(file_path_list), i, tp)
        i = i + 1
    plt.show() 

# show_registered_images('9939718R')

In [30]:
def display_sagittal_images(fixed_image, moving_image):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    fixed_npa = sitk.GetArrayViewFromImage(fixed_image)
    moving_npa = sitk.GetArrayViewFromImage(moving_image)
    size = np.size(fixed_npa, 2)
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
    plt.imshow(fixed_npa[:,:,size//2],cmap=plt.cm.Greys_r);
    plt.title('fixed image')
    plt.axis('off')
    
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
    plt.imshow(moving_npa[:,:,size//2],cmap=plt.cm.Greys_r);
    plt.title('moving image')
    plt.axis('off')
    
    plt.show() 